<a href="https://colab.research.google.com/github/jaredlan1/getting_started_with_ML/blob/main/HW_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from scipy.stats import qmc

np.random.seed(0)
torch.manual_seed(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cpu


In [10]:
debug = 1

In [25]:
nu = 0.01 / np.pi # viscosity
N_u = 100 # num of labeld points, Intial and Boundary
N_f = 10000 # numb of collocation points

# initial conditions : u(0,x) = -sin(pi*x)

N_ic = 50 # step size for x_ic
x_ic = np.linspace(-1,1, N_ic)[:, None] #[] transform to 2d column vector
if debug == 1 :print(f"x_ic : {x_ic}")
t_ic = np.zeros_like(x_ic)
u_ic = -np.sin(np.pi * x_ic)
if debug ==1 :print(f'u_ic : {u_ic}')

#BC
N_bc = N_u - N_ic
t_bc = np.random.rand(N_bc, 1)
x_bc = np.vstack([
        -np.ones((N_bc//2,1)),
        np.ones((N_bc - N_bc // 2,1))
])
u_bc = np.zeros_like(x_bc)

if debug ==1: print(f"x_bc : {x_bc}")

x_u = np.vstack([x_ic, x_bc])
t_u = np.vstack([t_ic, t_bc])
u_u = np.vstack([u_ic, u_bc])

if debug ==1: print(f"shape of x_u: {x_u.shape}") #(100,1)

#setting collocation poitns using the latinhypercube
sampler= qmc.LatinHypercube(d=2, seed=0)
X_f = sampler.random(n=N_f)
x_f = 2.0 * X_f[:, [0]] - 1.0 #scale to [-1,1]
t_f = X_f[:, [1]] #[0,1]

def to_tensor(a, requires_grad=False, dtype=torch.float32):
    return torch.tensor(a, dtype=dtype, device=device, requires_grad=requires_grad)

x_u_t = to_tensor(x_u, requires_grad=True)
t_u_t = to_tensor(t_u, requires_grad=True)
u_u_t = to_tensor(u_u, requires_grad=False)

x_f_all = to_tensor(x_f, requires_grad=False)
t_f_all = to_tensor(t_f, requires_grad=False)

if debug ==1 :print("x_u_t:", x_u_t.shape, "t_u_t:", t_u_t.shape, "u_u_t:", u_u_t.shape)
if debug ==1 :print("x_f_all:", x_f_all.shape, "t_f_all:", t_f_all.shape)


x_ic : [[-1.        ]
 [-0.95918367]
 [-0.91836735]
 [-0.87755102]
 [-0.83673469]
 [-0.79591837]
 [-0.75510204]
 [-0.71428571]
 [-0.67346939]
 [-0.63265306]
 [-0.59183673]
 [-0.55102041]
 [-0.51020408]
 [-0.46938776]
 [-0.42857143]
 [-0.3877551 ]
 [-0.34693878]
 [-0.30612245]
 [-0.26530612]
 [-0.2244898 ]
 [-0.18367347]
 [-0.14285714]
 [-0.10204082]
 [-0.06122449]
 [-0.02040816]
 [ 0.02040816]
 [ 0.06122449]
 [ 0.10204082]
 [ 0.14285714]
 [ 0.18367347]
 [ 0.2244898 ]
 [ 0.26530612]
 [ 0.30612245]
 [ 0.34693878]
 [ 0.3877551 ]
 [ 0.42857143]
 [ 0.46938776]
 [ 0.51020408]
 [ 0.55102041]
 [ 0.59183673]
 [ 0.63265306]
 [ 0.67346939]
 [ 0.71428571]
 [ 0.75510204]
 [ 0.79591837]
 [ 0.83673469]
 [ 0.87755102]
 [ 0.91836735]
 [ 0.95918367]
 [ 1.        ]]
u_ic : [[ 1.22464680e-16]
 [ 1.27877162e-01]
 [ 2.53654584e-01]
 [ 3.75267005e-01]
 [ 4.90717552e-01]
 [ 5.98110530e-01]
 [ 6.95682551e-01]
 [ 7.81831482e-01]
 [ 8.55142763e-01]
 [ 9.14412623e-01]
 [ 9.58667853e-01]
 [ 9.87181783e-01]
 [ 9.99

In [29]:
class PINN( torch.nn.Module):
  def __init__(self, layers):
    super().__init__()
    self.layers = torch.nn.ModuleList()
    for i in range(len(layers)-1):
      self.layers.append(torch.nn.Linear(layers[i], layers[i+1]))

      for m in self.layers:
          torch.nn.init.xavier_normal_(m.weight)
          torch.nn.init.zeros_(m.bias)

  def forward(self, x,t):
    z=torch.cat([x,t], dim=1)
    for i, layer in enumerate(self.layers):
      z=layer(z)
      if i < len(self.layers)-1:
          z=torch.tanh(z)
    return z

model = PINN([2,64,64,64,64,1]).to(device)
if debug ==1:print(model)

PINN(
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=64, bias=True)
    (1-3): 3 x Linear(in_features=64, out_features=64, bias=True)
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)
